# Cleaning database keys

### The challenge
Data scientists often rely on data retrieved from public databases. One issue particularly with big databases can be that there is no or insufficient quality control of the entered data. Here we work on a solution for the issues of different taxon names (due to misspellings or different taxonomy) in fossil databases, but the scripts can be applied to any database that is supposed to contain categorical data, yet contains inconsistent strings for many of these categories.




The fossil data in this project is retrieved from a set of online databases, namely the Paleobiology Database ([PBDB](https://paleobiodb.org/#/)), [NOW](http://www.helsinki.fi/science/now/) database, [Sahul](https://www.nature.com/articles/sdata201653) database and [Neotoma](https://www.neotomadb.org/) database. All databases were download in their entirety and the data was joined by extracting only the columns of interest, which are the following:

In [8]:
import tabletext

data = [['name','max_age','min_age','lat','lng','family','order','database','reference']]
print(tabletext.to_text(data))

┌──────┬─────────┬─────────┬─────┬─────┬────────┬───────┬──────────┬───────────┐
│ name │ max_age │ min_age │ lat │ lng │ family │ order │ database │ reference │
└──────┴─────────┴─────────┴─────┴─────┴────────┴───────┴──────────┴───────────┘


###### The fossil data file looks like this:

In [7]:
%%bash
#skipping the reference column since it is very messy
head -n 10 data/joined_databases.txt | cut -f 1-8

name	max_age	min_age	lat	lng	family	order	database
Aaptorcytes ivyi	57.7	57.3	44.678056	-106.978889	Palaeoryctidae	Cimolesta	now
Aaptorcytes ivyi	57.3	56.3	44.678056	-106.978889	Palaeoryctidae	Cimolesta	now
Aaptoryctes ivyi	61.7	56.8	44.693604	-108.3554	Palaeoryctidae	Cimolesta	pbdb
Aaptoryctes ivyi	56.8	55.8	44.299999	-109.0	Palaeoryctidae	Cimolesta	pbdb
Aaptoryctes ivyi	61.7	56.8	42.700001	-110.099998	Palaeoryctidae	Cimolesta	pbdb
Abderites meridionalis	17.5	16.3	-48.516666	-69.716667	Abderitidae	Paucituberculata	pbdb
Abderites meridionalis	16.3	15.5	-44.577498999999996	-71.220833	Abderitidae	Paucituberculata	pbdb
Abderites meridionalis	23.03	15.97	-45.666668	-68.666664	Abderitidae	Paucituberculata	pbdb
Abderites meridionalis	17.5	16.3	-50.599998	-69.083336	Abderitidae	Paucituberculata	pbdb


###### The issue with the names in this database is that there are some misspellings, e.g.:

In [6]:
%%bash
sed '1045q;d' data/joined_databases.txt | cut -f 1-8
sed '1046q;d' data/joined_databases.txt | cut -f 1-8

Adunator fredericki	63.3	61.7	41.301398999999996	-107.735703		Macroscelidea	pbdb
Adunator fredricki	61.3	60.9	41.14555	-104.80194399999999	Litocheridae	Eulipotyphla	now


###### If we were to categorize the data, these would end up being two different taxa, even though they are both occurrences of the same species. We therefore need to join cases like this into the same category by clustering similar textstrings. However this clustering has to be done carfully since there are also cases where two different taxa names have very similar spellings and differ in only two letters as e.g.:


In [5]:
%%bash
sed '977q;d' data/joined_databases.txt | cut -f 1-8
sed '978q;d' data/joined_databases.txt | cut -f 1-8

Adjidaumo maximus	33.9	33.3	42.400002	-103.800003	Eomyidae	Rodentia	pbdb
Adjidaumo minimus	36.5	35.7	43.029722	-106.80916699999999	Eomyidae	Rodentia	now


###### These names should not be clustered but kept separately, as they represent two legitimate different categories (species).

### Solution
First, we calculate the pairwise distance between any pair of strings in the taxon list and store these in a similarity matrix, using a python function, which is stored in the bin directory:

In [1]:
import sys
sys.path
sys.path.append("./bin")
import create_simmatrix
import pandas as pd
from scipy.io import mmwrite

fossil_file = pd.read_csv('/Users/tobias/GitHub/correct_database_keys/data/joined_databases.txt',sep='\t',low_memory=False)

#reduce the original dataset to the first 1050 lines, for faster computation
sample_set_fossil_file = fossil_file.loc[:1050]
taxon_names = list(sample_set_fossil_file.name)

matrix = create_simmatrix.create_simmatrix(taxon_names)

# write to Matrix Market format for passing to DBSCAN
#mmwrite('./intermediate_files/taxon_sim_names.mtx', matrix)


Processed 100/1051 rows of data
Processed 200/1051 rows of data
Processed 300/1051 rows of data
Processed 400/1051 rows of data
Processed 500/1051 rows of data
Processed 600/1051 rows of data
Processed 700/1051 rows of data
Processed 800/1051 rows of data
Processed 900/1051 rows of data
Processed 1000/1051 rows of data


In [2]:
import smart_clustering
from IPython.display import display, HTML
from scipy.io import mmread

#matrix = mmread('./taxon_sim_names.mtx')

clusters = smart_clustering.smart_cluster_based_on_matrix(matrix,taxon_names)

display(clusters.head(10))
#clusters.head(10).to_html()

clusters.to_csv('/Users/tobias/GitHub/correct_database_keys/intermediate_files/clusters.txt', sep = '\t', index = False, header=False)
with open('/Users/tobias/GitHub/correct_database_keys/intermediate_files/clusters.txt', 'a') as file:
    file.write('Original number of categories: %i' %len(set(taxon_names)))


,0
0,"Cluster#0: Aaptorcytes ivyi, Aaptorcytes ivyi,..."
1,"Cluster#1: Abderites meridionalis, Abderites m..."
2,Cluster#2: Abdounodus hamdii
3,Cluster#3: Abelmoschomys phthanus
4,"Cluster#4: Abelmoschomys simpsoni, Abelmoschom..."
5,"Cluster#5: Abelmoschomys valensis, Abelmoschom..."
6,Cluster#6: Abelmoschomys vasquezi
7,"Cluster#7: Abeomelomys sevia, Abeomelomys sevi..."
8,"Cluster#8: Aboletylestes hypselus, Aboletylest..."
9,Cluster#9: Aboletylestes robustus


###### This clustering, based on a neuronal network algorithm, appears to take care of the issues. Different variations of the same taxon name base don mis-spellings are correctly joined in the same cluster,

In [3]:
%%bash
sed '153q;d' intermediate_files/clusters.txt
sed '141q;d' intermediate_files/clusters.txt

Cluster#152: Adunator fredericki, Adunator fredricki
Cluster#140: Adilophontes brachykolos, Adilophontes brachykolos, Adilophontes brachykolos, Adilophontes brackykolos


###### ... while taxon names that are very similar and only differ in a few characters are correctly sorted into separate clusters.

In [4]:
%%bash
sed '149q;d' intermediate_files/clusters.txt
sed '150q;d' intermediate_files/clusters.txt
sed '151q;d' intermediate_files/clusters.txt

Cluster#148: Adjidaumo maximus, Adjidaumo maximus, Adjidaumo maximus, Adjidaumo maximus, Adjidaumo maximus
Cluster#149: Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus, Adjidaumo minimus
Cluster#150: Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo minutus, Adjidaumo min

Besides spelling there is another issue, which has to do with the taxonomy for fossils changing through time, as new fossils are being uncovered, which sometimes leads to a taxon being assigned to a different genus. This introduces incosistencies in the databases, since different fossils of the same taxon may have been described at different times, using different genus names for the same taxon. Further there is much debate and disagreement in the paleoontological community, leading to different paleontologists sticking to different taxonomies.
But there is some light at the end of the tunnel: Based on the genus epithet (which always stays consistent) and the information about the author and year of the first description of the taxon, we can resolve many of these taxonomic inconsistencies.

Since the information of author and year of first description is not available in most fossil occurrence databases, we downloaded this information from the [Zoological Record database](http://wokinfo.com/products_tools/specialized/zr/), which contains a list of most described animal species, including the author and year of first description. This database also contains many variations of species names found in the literature (some of which are legitimate synonyms, some of which are spelling errors or disagreements). However, these synonyms and spelling incosistencies are not linked to each other. Therefore we apply the same fuzzy clustering algorithm as used on the fossil occurrence data above: